In [15]:
import os
from pathlib import Path
import getpass
import numpy as np
import time
import torch
import PIL
from torch import nn
from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader
import random
import sys

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.argparser import parser
from src.model.dino_model import get_dino, ViTWrapper
from src.model.data import *

# Custom imports
import torchattacks
from torchattacks import *
import torch.optim as optim
from torchvision import transforms as pth_transforms
from torchvision.utils import save_image

from torchvision.models.resnet import ResNet, Bottleneck

In [18]:
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels)
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)
    
class CustomResNet(ResNet):
    def __init__(self, classifier=None):
        super(CustomResNet, self).__init__(block=Bottleneck, layers=[3, 4, 6, 3])
        self.load_state_dict(torch.load("/cluster/scratch/jrando/resnet/resnet.pth"))
        del self.fc
        
        self.classifier = classifier
        
    def _forward_impl(self, x):
        # Normalize
        transform = pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        x = transform(x)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        if self.classifier:
            x = self.classifier(x)

        return x



In [22]:
DATA = Path('/cluster/scratch/thobauma/data/')
ORI = DATA/'ori/filtered'

In [64]:
TRAIN_PATH = ORI/'train'
VALIDATION_PATH = ORI/'validation'

linear_classifier = LinearClassifier(2048, 9)
linear_classifier.load_state_dict(torch.load("/cluster/scratch/thobauma/data/models/resnet/checkpoint.pth.tar")['state_dict'])
linear_classifier.to('cuda')

model = CustomResNet(classifier=linear_classifier).to('cuda')

atk = PGD(model, eps=0.01, alpha=(0.001*2)/3, steps=3)
name='pgd_01'

STORE_PATH = Path('/cluster/scratch/jrando/delete', 'adv', name)

print('Validation set')

val_dataset = AdvTrainingImageDataset(VALIDATION_PATH/'images', 
                              VALIDATION_PATH/'labels.csv', 
                              ORIGINAL_TRANSFORM, index_subset=[1000])

val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


STORE_LABEL_PATH = Path(STORE_PATH, 'validation', 'labels.csv')
STORE_LABEL_PATH.parent.mkdir(parents=True, exist_ok=True)
STORE_IMAGES_PATH = Path(STORE_PATH, 'validation', 'images')
STORE_IMAGES_PATH.mkdir(parents=True, exist_ok=True)

correct = 0
start = time.time()
true_labels = []
adv_labels = []
names = []
store_images = []

print(f'''saving predictions to: {STORE_LABEL_PATH}''')
print(f'''saving output tensors to: {STORE_IMAGES_PATH}\n''')

for images, labels, img_names in tqdm(val_loader):
    images.cuda(non_blocking=True)
    labels = labels.cuda(non_blocking=True)

    adv_images = atk(images, labels)
    store_images.append(adv_images)

    with torch.no_grad():
        outputs = model(adv_images)

    _, pre = torch.max(outputs.data, 1)
    print(pre)
    print(labels)

    correct += (pre == labels).sum()

    for adv_img, img_name in zip(adv_images, img_names):
        print(f"saving at {Path(STORE_IMAGES_PATH, img_name)}")
        save_image(adv_img, fp=Path(STORE_IMAGES_PATH, img_name), format= "PNG")

    true_labels.extend(labels.detach().cpu().tolist())
    adv_labels.extend(pre.detach().cpu().tolist())
    names.extend(img_names)

print('Total elapsed time (sec): %.2f' % (time.time() - start))
print('Accuracy against attack: %.2f %%' % (100 * float(correct) / len(val_loader.dataset)))

data_dict = {'image': names, 'reduced_label':true_labels, name+'_pred':adv_labels}
df = pd.DataFrame(data_dict)
df['image'] = df['image'].str.split('.').str[0]
df.to_csv(STORE_LABEL_PATH, sep=",")
print(f'''\n''')

100%|██████████| 1/1 [00:00<00:00,  8.54it/s]

Validation set
saving predictions to: /cluster/scratch/jrando/delete/adv/pgd_01/validation/labels.csv
saving output tensors to: /cluster/scratch/jrando/delete/adv/pgd_01/validation/images

tensor([0], device='cuda:0')
tensor([2], device='cuda:0')
saving at /cluster/scratch/jrando/delete/adv/pgd_01/validation/images/ILSVRC2012_val_00031506
Total elapsed time (sec): 0.12
Accuracy against attack: 0.00 %




In [62]:
import pickle
with open("./test.tch", "wb") as f:
    torch.save(store_images[0], f)

In [36]:
store_images[0]

tensor([[[[0.6438, 0.6368, 0.6346,  ..., 0.5662, 0.5248, 0.5582],
          [0.6324, 0.6245, 0.6232,  ..., 0.5967, 0.5624, 0.5740],
          [0.6143, 0.6273, 0.6400,  ..., 0.6794, 0.6571, 0.6290],
          ...,
          [0.7486, 0.6650, 0.7241,  ..., 0.7032, 0.7551, 0.7406],
          [0.7795, 0.7402, 0.7555,  ..., 0.8044, 0.8162, 0.7921],
          [0.8449, 0.8394, 0.8566,  ..., 0.6850, 0.7608, 0.7440]],

         [[0.5090, 0.5128, 0.4981,  ..., 0.4303, 0.4243, 0.4477],
          [0.4896, 0.4817, 0.4798,  ..., 0.4732, 0.4364, 0.4574],
          [0.4757, 0.4730, 0.4896,  ..., 0.5701, 0.5355, 0.5110],
          ...,
          [0.6368, 0.5626, 0.6127,  ..., 0.6093, 0.6626, 0.6714],
          [0.6819, 0.6292, 0.6494,  ..., 0.6980, 0.7133, 0.6917],
          [0.7620, 0.7503, 0.7666,  ..., 0.5553, 0.6292, 0.6341]],

         [[0.5105, 0.5047, 0.4841,  ..., 0.4274, 0.4178, 0.4316],
          [0.5004, 0.4853, 0.4833,  ..., 0.4658, 0.4410, 0.4343],
          [0.4622, 0.4731, 0.4859,  ..., 0

In [ ]:
torch.save()

In [65]:
img = Image.open("/cluster/scratch/jrando/delete/adv/pgd_01/validation/images/ILSVRC2012_val_00031506")
img = img.convert('RGB')

In [66]:
ONLY_NORMALIZE_TRANSFORM = pth_transforms.Compose([
                                            pth_transforms.ToTensor(),
                                            ])

In [67]:
test = ONLY_NORMALIZE_TRANSFORM(img)

In [69]:
model(test.unsqueeze(0).cuda())

tensor([[ 1.7783,  0.6789,  0.6403,  0.3051, -0.2316,  0.3395, -0.6651, -0.3711,
         -0.8962]], device='cuda:0', grad_fn=<AddmmBackward>)

In [63]:
!cp /cluster/scratch/jrando/delete/adv/pgd_01/validation/images/ILSVRC2012_val_00032854 ./test_ong